# Imports

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

from xgboost import XGBRegressor


pd.set_option('display.float_format', '{:.2f}'.format)

df = pd.read_csv("https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv")

# Data Exploration

## Evaluating Outliers

In [21]:
df

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
0,1565930130,20141104T000000,4,3.25,3760,4675,2.00,0,0,3,...,2740,1020,2007,0,98038,47.39,-122.05,3280,4033,429900.00
1,3279000420,20150115T000000,3,1.75,1460,7800,1.00,0,0,2,...,1040,420,1979,0,98023,47.30,-122.38,1310,7865,233000.00
2,194000575,20141014T000000,4,1.00,1340,5800,1.50,0,2,3,...,1340,0,1914,0,98116,47.57,-122.39,1900,5800,455000.00
3,2115510160,20141208T000000,3,1.75,1440,8050,1.00,0,0,3,...,1440,0,1985,0,98023,47.32,-122.39,1790,7488,258950.00
4,7522500005,20140815T000000,2,1.50,1780,4750,1.00,0,0,4,...,1080,700,1947,0,98117,47.69,-122.39,1690,5962,555000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,5272200045,20141113T000000,3,1.50,1000,6914,1.00,0,0,3,...,1000,0,1947,0,98125,47.71,-122.32,1000,6947,378000.00
19996,9578500790,20141111T000000,3,2.50,3087,5002,2.00,0,0,3,...,3087,0,2014,0,98023,47.30,-122.35,2927,5183,399950.00
19997,7202350480,20140930T000000,3,2.50,2120,4780,2.00,0,0,3,...,2120,0,2004,0,98053,47.68,-122.03,1690,2650,575000.00
19998,1723049033,20140620T000000,1,0.75,380,15000,1.00,0,0,3,...,380,0,1963,0,98168,47.48,-122.32,1170,15000,245000.00


In [22]:
# Graphing out the distribution of various house sizes
sqft_count = (df["sqft_living"].floordiv(100)*100).value_counts().reset_index().sort_values("sqft_living")

fig1 = px.line(
    sqft_count,
    x="sqft_living",
    y="count"
)
fig1.show()

In [23]:
# Graphing out the distribution of various house sizes
lot_count = (df["sqft_lot"].floordiv(100)*100).value_counts().reset_index().sort_values("sqft_lot")

fig2 = px.line(
    lot_count,
    x="sqft_lot",
    y="count"
)
fig2.show()

In [24]:
# Graphing it again but excluding 
excluding_amount = 45000

lot_count = (df[df["sqft_lot"]<excluding_amount]["sqft_lot"].floordiv(500)*500).value_counts().reset_index().sort_values("sqft_lot")

fig2 = px.line(
    lot_count,
    x="sqft_lot",
    y="count"
)
fig2.show()

display(df[df["sqft_lot"]>excluding_amount])

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
56,2195700050,20140519T000000,4,2.50,3480,59242,2.00,0,0,3,...,3480,0,1988,0,98072,47.74,-122.10,2930,39400,810000.00
63,2423069164,20150410T000000,3,2.00,1990,65340,2.00,0,0,3,...,1990,0,1986,0,98027,47.47,-121.99,2120,59241,500000.00
77,522069119,20150512T000000,3,2.50,2720,62310,1.00,0,0,3,...,2040,680,1985,0,98038,47.42,-122.07,2770,204296,550000.00
79,826069184,20141002T000000,3,2.50,1960,47044,2.00,0,0,4,...,1960,0,1978,0,98077,47.76,-122.07,2020,29004,535000.00
92,824069173,20140821T000000,3,2.50,2320,52272,1.50,0,0,3,...,2320,0,1974,0,98075,47.59,-122.07,2200,52272,600000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19872,1523069151,20140711T000000,2,1.00,1470,81021,1.00,0,0,4,...,1470,0,1949,0,98027,47.48,-122.03,2600,69696,380000.00
19878,1823099056,20141222T000000,3,2.50,2810,435600,2.00,0,0,3,...,2810,0,1995,0,98045,47.48,-121.70,2380,92007,745000.00
19936,9510320150,20141202T000000,4,2.50,2500,50595,2.00,0,0,3,...,2500,0,1997,0,98045,47.47,-121.73,2765,33720,545000.00
19958,1522039105,20150115T000000,3,4.25,3300,308080,2.00,0,2,4,...,2520,780,1976,0,98070,47.40,-122.42,2130,90604,729000.00


In [25]:
price_count = (df["price"].floordiv(10000)*10000).value_counts().reset_index().sort_values("price")

fig2 = px.line(
    price_count,
    x="price",
    y="count"
)
fig2.show()

In [26]:
df[df["price"]>1500000]

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
13,251500330,20141014T000000,3,2.25,4320,19225,1.00,0,0,4,...,2160,2160,1972,0,98004,47.64,-122.22,3430,18469,1775000.00
88,1224059053,20141027T000000,5,2.00,2500,15250,2.00,1,4,5,...,2500,0,1942,0,98008,47.59,-122.11,1880,18782,1700000.00
96,1068000255,20140827T000000,4,3.50,4285,9567,2.00,0,1,5,...,3485,800,1946,0,98199,47.64,-122.41,2960,6902,1650000.00
203,9412400220,20140710T000000,4,2.75,5470,18200,2.00,1,4,3,...,3730,1740,1992,0,98118,47.53,-122.26,3620,15100,1612500.00
231,9315600050,20150317T000000,5,3.25,4560,19080,1.00,0,0,5,...,2490,2070,1963,0,98004,47.63,-122.23,3390,20140,1675000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19886,3893100462,20150225T000000,5,3.75,4360,8504,2.00,0,4,3,...,3530,830,2014,0,98033,47.69,-122.19,2680,9000,1789950.00
19895,4139910160,20150401T000000,5,3.25,4320,32840,2.00,0,0,3,...,4320,0,1990,0,98006,47.55,-122.12,4410,33210,1600000.00
19916,3885803044,20140902T000000,4,5.00,5810,7440,2.00,0,0,3,...,3790,2020,2004,0,98033,47.69,-122.21,3010,7200,1875000.00
19938,9362000040,20140623T000000,3,4.50,5230,17826,2.00,1,4,3,...,3740,1490,2005,0,98040,47.53,-122.24,3670,17826,3400000.00


In [27]:
yr = df["yr_built"].value_counts().reset_index().sort_values("yr_built")

fig5 = px.bar(
    yr,
    x="yr_built",
    y="count"
)
fig5.show()

In [28]:
df["yr_sold"] = df["date"].str[0:4].to_numpy(np.int64)
df["yr_sold"].value_counts()

yr_sold
2014    13574
2015     6426
Name: count, dtype: int64

In [29]:
duplicates = df[df["id"].duplicated(keep=False)].sort_values("id")

In [30]:
duplicates

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price,yr_sold
10022,1000102,20140916T000000,6,3.00,2400,9373,2.00,0,0,3,...,0,1991,0,98002,47.33,-122.21,2060,7316,280000.00,2014
1272,1000102,20150422T000000,6,3.00,2400,9373,2.00,0,0,3,...,0,1991,0,98002,47.33,-122.21,2060,7316,300000.00,2015
2124,7200179,20150424T000000,2,1.00,840,12750,1.00,0,0,3,...,0,1925,0,98055,47.48,-122.21,1480,6969,175000.00,2015
11317,7200179,20141016T000000,2,1.00,840,12750,1.00,0,0,3,...,0,1925,0,98055,47.48,-122.21,1480,6969,150000.00,2014
2103,109200390,20140820T000000,3,1.75,1480,3900,1.00,0,0,4,...,0,1980,0,98023,47.30,-122.37,1830,6956,245000.00,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15385,9809000020,20140513T000000,5,2.25,3120,16672,2.00,0,0,4,...,0,1969,0,98004,47.65,-122.22,3740,17853,1895000.00,2014
13013,9834200305,20140716T000000,3,1.00,1790,3876,1.50,0,0,5,...,700,1904,0,98144,47.58,-122.29,1360,4080,350000.00,2014
17200,9834200305,20150210T000000,3,1.00,1790,3876,1.50,0,0,5,...,700,1904,0,98144,47.58,-122.29,1360,4080,615000.00,2015
3817,9834200885,20140717T000000,4,2.50,2080,4080,1.00,0,0,5,...,1040,1962,0,98144,47.57,-122.29,1340,4080,360000.00,2014


# testing models

In [31]:
df = df[df["price"]< 2000000]


# Base, no cleaning of data or parameter testing.
X = pd.get_dummies(df.drop(["price", "date"], axis=1))
X["yr_sold"] = df["date"].str[0:4].to_numpy(np.int64)
X["mnth_sold"] = df["date"].str[4:6].to_numpy(np.int64)
y = df["price"]

model = XGBRegressor()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

sse = np.sum((y_test - y_pred) ** 2)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

display(sse)
display(mse)
display(rmse)
display(r2)



37710962537109.78

9515761427.481651

97548.76435650865

0.8807028756629424

In [32]:
mini = pd.read_csv("https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing_holdout_test_mini.csv")

X = pd.get_dummies(mini.drop(["date"], axis=1))
X["yr_sold"] = mini["date"].str[0:4].to_numpy(np.int64)
X["mnth_sold"] = mini["date"].str[4:6].to_numpy(np.int64)

X

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,yr_sold,mnth_sold
0,930000305,4,1.75,2120,7680,1.00,0,0,4,7,...,1060,1950,0,98177,47.72,-122.36,1530,7680,2014,11
1,9541600015,4,2.25,2010,15375,1.00,0,0,4,8,...,0,1957,0,98005,47.60,-122.17,1930,15375,2015,2
2,7338000150,2,1.00,1070,4200,1.00,0,0,4,6,...,0,1983,0,98002,47.33,-122.22,1150,4200,2015,1
3,6113400046,4,2.50,1890,15770,2.00,0,0,4,7,...,0,1968,0,98166,47.43,-122.34,2410,15256,2014,7
4,291310170,3,2.50,1600,2610,2.00,0,0,3,8,...,0,2005,0,98027,47.53,-122.07,1445,1288,2014,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,8902000267,4,2.75,2260,7209,1.00,0,3,3,7,...,930,2002,0,98125,47.71,-122.30,1790,10860,2015,4
77,7856550240,5,2.25,3480,9200,2.00,0,0,3,8,...,0,1979,0,98006,47.56,-122.15,3130,9200,2014,7
78,7923500060,5,2.75,2580,9242,2.00,0,2,4,8,...,860,1967,0,98007,47.59,-122.13,2240,9316,2014,9
79,8898700880,2,2.00,1590,8000,1.00,0,0,3,7,...,680,1984,0,98055,47.46,-122.20,1590,8364,2015,3


In [33]:

mini_pred = model.predict(X)
mini_pred = pd.DataFrame(mini_pred, columns = ['price'])
mini_pred.to_csv("Joseph-predictions.csv", index=False)




# Final Test

In [5]:
import numpy as np
import pandas as pd
import plotly.express as px

from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error
from sklearn.model_selection import train_test_split
from geopy.distance import geodesic

from xgboost import XGBRegressor

pd.set_option('display.float_format', '{:.2f}'.format)
df = pd.read_csv("https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv")
df = df[df["price"]<4000000]
# df = df[df["long"] < -121.6]

# Downtown Seattle's coordinates
downtown_seattle = (47.6062, -122.3321)
mt_rain = (46.8523, -121.7603)

# Function to calculate distance from downtown Seattle
def calculate_distance(lat, long, loc):
    return geodesic((lat, long), loc).miles  # Or .km for kilometers

# Apply the function to each row in the dataset
# df['dist_downtown'] = df.apply(lambda row: calculate_distance(row['lat'], row['long'], downtown_seattle), axis=1)
# df['dist_mt_rain'] = df.apply(lambda row: calculate_distance(row['lat'], row['long'], mt_rain), axis=1)







# df = df[df["price"]< 2000000]

# Base, no cleaning of data or parameter testing.
X = pd.get_dummies(df.drop(["price", "date"], axis=1))
X["yr_sold"] = df["date"].str[0:4].to_numpy(np.int64)
X["mnth_sold"] = df["date"].str[4:6].to_numpy(np.int64)
y = df["price"]

model = XGBRegressor()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

model.fit(X_train, y_train)

pred = model.predict(X_test)

display(root_mean_squared_error(y_test, pred))


mini = pd.read_csv("https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing_holdout_test_mini.csv")

# mini['dist_downtown'] = mini.apply(lambda row: calculate_distance(row['lat'], row['long'], downtown_seattle), axis=1)
# mini['dist_mt_rain'] = mini.apply(lambda row: calculate_distance(row['lat'], row['long'], mt_rain), axis=1)

X = pd.get_dummies(mini.drop(["date"], axis=1))
X["yr_sold"] = mini["date"].str[0:4].to_numpy(np.int64)
X["mnth_sold"] = mini["date"].str[4:6].to_numpy(np.int64)


mini_pred = model.predict(X)
mini_pred = pd.DataFrame(mini_pred, columns = ['price'])
mini_pred.to_csv("Team2-J+I-predictions.csv", index=False)


112464.05346905472

In [46]:
pred

array([848032.1 , 807892.3 , 793479.6 , ..., 469769.38, 438626.94,
       470219.4 ], dtype=float32)

# Layout

In [39]:
df = df[df["long"] < -121.6]
# df = df[df["price"] < ]


figx = px.scatter(
    df,
    x="long",
    y="lat",
    color="price",
    color_continuous_scale=[(0, "green"), (.01, "green"),(.05, "yellow"), (.2, "red"), (1, "blue")],
    # hover_data="dist_downtown"
    labels={"lat":"Latitude", "long":"Longitude", "price":"Price"},
    title="Map of Price"
    
)


# Update layout to make x and y axes scale evenly
figx.update_layout(
    yaxis=dict(scaleanchor="x", scaleratio=1),  # Ensure equal scaling between x and y
    xaxis=dict(scaleanchor="y", scaleratio=1)
)


figx.add_annotation(
    x=df["long"].mean() - 0.1,  # Position the arrow at the mean of longitude
    y=df["lat"].mean() + .05,    # Position the arrow at the mean of latitude
    axref="x",  # Reference the x-axis for the arrow start
    ayref="y",  # Reference the y-axis for the arrow start
    ax=df["long"].mean() - 0.4,  # Set where the arrow should start (adjust accordingly)
    ay=df["lat"].mean() - 0.1,   # Set where the arrow should start (adjust accordingly)
    text="Seattle",  # Label for the annotation
    showarrow=True,  # Display the arrow
    arrowhead=2,     # Style of the arrowhead
    arrowsize=1.5,   # Size of the arrow
    arrowwidth=2,    # Width of the arrow line
    arrowcolor="black"  # Color of the arrow
)

# Display the figure
figx.show()

In [26]:
mini

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,930000305,20141110T000000,4,1.75,2120,7680,1.00,0,0,4,7,1060,1060,1950,0,98177,47.72,-122.36,1530,7680
1,9541600015,20150211T000000,4,2.25,2010,15375,1.00,0,0,4,8,2010,0,1957,0,98005,47.60,-122.17,1930,15375
2,7338000150,20150129T000000,2,1.00,1070,4200,1.00,0,0,4,6,1070,0,1983,0,98002,47.33,-122.22,1150,4200
3,6113400046,20140723T000000,4,2.50,1890,15770,2.00,0,0,4,7,1890,0,1968,0,98166,47.43,-122.34,2410,15256
4,291310170,20140804T000000,3,2.50,1600,2610,2.00,0,0,3,8,1600,0,2005,0,98027,47.53,-122.07,1445,1288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,8902000267,20150402T000000,4,2.75,2260,7209,1.00,0,3,3,7,1330,930,2002,0,98125,47.71,-122.30,1790,10860
77,7856550240,20140710T000000,5,2.25,3480,9200,2.00,0,0,3,8,3480,0,1979,0,98006,47.56,-122.15,3130,9200
78,7923500060,20140922T000000,5,2.75,2580,9242,2.00,0,2,4,8,1720,860,1967,0,98007,47.59,-122.13,2240,9316
79,8898700880,20150317T000000,2,2.00,1590,8000,1.00,0,0,3,7,910,680,1984,0,98055,47.46,-122.20,1590,8364


In [28]:
df.describe()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
count,19988.00,19988.00,19988.00,19988.00,19988.00,19988.00,19988.00,19988.00,19988.00,19988.00,19988.00,19988.00,19988.00,19988.00,19988.00,19988.00,19988.00,19988.00,19988.00,19988.00
mean,4579445981.22,3.37,2.11,2076.04,15007.11,1.50,0.01,0.23,3.41,7.65,1787.41,288.63,1971.09,82.48,98077.90,47.56,-122.21,1985.27,12743.69,536516.46
std,2875114218.67,0.93,0.76,905.35,39711.21,0.54,0.09,0.76,0.65,1.17,821.23,438.54,29.37,397.29,53.44,0.14,0.14,683.75,27082.95,345460.08
min,1000102.00,0.00,0.00,290.00,520.00,1.00,0.00,0.00,1.00,1.00,290.00,0.00,1900.00,0.00,98001.00,47.16,-122.52,399.00,651.00,75000.00
25%,2123049363.50,3.00,1.75,1430.00,5040.00,1.00,0.00,0.00,3.00,7.00,1200.00,0.00,1951.00,0.00,98033.00,47.47,-122.33,1490.00,5100.00,321837.50
50%,3904940180.00,3.00,2.25,1910.00,7620.00,1.50,0.00,0.00,3.00,7.00,1560.00,0.00,1975.00,0.00,98065.00,47.57,-122.23,1840.00,7620.00,450000.00
75%,7308950385.00,4.00,2.50,2550.00,10696.25,2.00,0.00,0.00,4.00,8.00,2210.00,557.00,1997.00,0.00,98117.00,47.68,-122.12,2360.00,10080.00,641000.00
max,9900000190.00,33.00,8.00,13540.00,1164794.00,3.50,1.00,4.00,5.00,13.00,9410.00,4130.00,2015.00,2015.00,98199.00,47.78,-121.31,6210.00,871200.00,3850000.00


In [14]:
from geopy.distance import geodesic


# Downtown Seattle's coordinates
downtown_seattle = (47.6062, -122.3321)
mt_rain = (46.8523, -121.7603)

# Function to calculate distance from downtown Seattle
def calculate_distance(lat, long):
    return geodesic((lat, long), downtown_seattle).miles  # Or .km for kilometers

def calc_dist_mt(lat, long):
    return geodesic((lat, long), mt_rain).miles  # Or .km for kilometers

# Apply the function to each row in the dataset
df['distance_from_downtown'] = df.apply(lambda row: calculate_distance(row['lat'], row['long']), axis=1)
df['distance_from_downtown'] = df.apply(lambda row: calc_dist_mt(row['lat'], row['long']), axis=1)



In [10]:
df["zipcode"].value_counts()

zipcode
98103    563
98038    550
98052    536
98115    530
98117    521
        ... 
98102     98
98010     90
98024     78
98148     49
98039     43
Name: count, Length: 70, dtype: int64

In [17]:
import numpy as np
import pandas as pd
import plotly.express as px

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from geopy.distance import geodesic

from xgboost import XGBRegressor



# Downtown Seattle's coordinates
downtown_seattle = (47.6062, -122.3321)
mt_rain = (46.8523, -121.7603)

# Function to calculate distance from downtown Seattle
def calculate_distance(lat, long, loc):
    return geodesic((lat, long), loc).miles  # Or .km for kilometers

# Apply the function to each row in the dataset
df['dist_downtown'] = df.apply(lambda row: calculate_distance(row['lat'], row['long'], downtown_seattle), axis=1)
df['dist_mt_rain'] = df.apply(lambda row: calculate_distance(row['lat'], row['long'], mt_rain), axis=1)

df[["dist_downtown", "dist_mt_rain"]]

,dist_downtown,dist_mt_rain
0,20.20,39.30
1,21.04,42.80
2,3.86,57.49
3,20.05,43.83
4,6.24,64.86
...,...,...
19995,7.50,65.09
19996,21.35,41.44
19997,14.93,58.66
19998,8.66,50.88
